# 03 - Optuna Optimization

Optimize strategy parameters using Optuna.

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import yaml
from pathlib import Path
import optuna

from optimization import OptunaOptimizer, ParamSpace, ObjectiveFunctions
from backtest import BacktestEngine, Signal

In [ ]:
# Load configuration
config_path = Path('../config/default.yaml')
with open(config_path) as f:
    config = yaml.safe_load(f)

print(f"Optimization config:")
print(f"  n_trials: {config['optimization']['n_trials']}")
print(f"  study_name: {config['optimization']['study_name']}")
print(f"  primary objective: {config['optimization']['objectives']['primary']}")

In [ ]:
# Load data
SYMBOL = config['market']['symbol']
features_path = Path('../data/processed') / f'{SYMBOL}_features.parquet'

if features_path.exists():
    data = pd.read_parquet(features_path)
else:
    # Create synthetic data
    print("Creating synthetic data...")
    n = 5000
    np.random.seed(42)
    returns = np.random.randn(n) * 0.001
    prices = 50000 * np.exp(np.cumsum(returns))
    
    data = pd.DataFrame({
        'open': prices * (1 + np.random.randn(n) * 0.0001),
        'high': prices * (1 + np.abs(np.random.randn(n) * 0.0005)),
        'low': prices * (1 - np.abs(np.random.randn(n) * 0.0005)),
        'close': prices,
        'volume': np.random.randint(10, 100, n) * 0.1,
        'rsi': 50 + np.random.randn(n) * 15,
        'atr': prices * 0.02,
    }, index=pd.date_range(start='2024-01-01', periods=n, freq='1min'))

print(f"Data shape: {data.shape}")

## Parameter Space

In [ ]:
# Check parameter space
param_space = ParamSpace(config)

print(f"Total optimizable parameters: {param_space.get_param_count()}")
print("\nParameter sections:")
for section in list(param_space.params.keys())[:10]:
    print(f"  {section}: {len(param_space.params[section])} params")

In [ ]:
# Get default parameters
defaults = param_space.get_default_params()
print("Sample default parameters:")
for k, v in list(defaults.items())[:10]:
    print(f"  {k}: {v}")

## Define Strategy

In [ ]:
def create_strategy(params):
    """
    Create strategy function with given parameters.
    """
    # Extract relevant parameters
    rsi_overbought = params.get('features.technical.rsi.overbought', 70)
    rsi_oversold = params.get('features.technical.rsi.oversold', 30)
    
    def strategy(features, context):
        rsi = features.get('rsi', 50)
        
        # Simple RSI strategy
        if rsi < rsi_oversold:
            return Signal(direction=1, confidence=0.7)  # Long
        elif rsi > rsi_overbought:
            return Signal(direction=-1, confidence=0.7)  # Short
        else:
            return Signal(direction=0)  # Hold
    
    return strategy

## Run Optimization

In [ ]:
# Initialize optimizer
optimizer = OptunaOptimizer(config)
backtest_engine = BacktestEngine(config)

In [ ]:
def objective(trial):
    """
    Optuna objective function.
    """
    # Suggest parameters
    params = {
        'features.technical.rsi.overbought': trial.suggest_int('rsi_overbought', 65, 80),
        'features.technical.rsi.oversold': trial.suggest_int('rsi_oversold', 20, 35),
    }
    
    # Create strategy
    strategy = create_strategy(params)
    
    # Run backtest
    results = backtest_engine.run(data, strategy, data)
    
    # Return Sharpe ratio
    return results.get('sharpe_ratio', 0.0)

In [ ]:
# Create study and optimize
study = optimizer.create_study(
    study_name='rsi_optimization',
    storage=None,  # In-memory for demo
)

# Run optimization (reduced trials for demo)
n_trials = 20  # Use config value for full run
best_params = optimizer.optimize(
    objective,
    n_trials=n_trials,
    show_progress_bar=True,
)

print(f"\nBest parameters: {best_params}")
print(f"Best Sharpe: {optimizer.get_best_value():.4f}")

In [ ]:
# Visualization
try:
    import optuna.visualization as vis
    
    fig = vis.plot_optimization_history(study)
    fig.show()
except:
    print("Visualization requires plotly")

In [ ]:
# Get trials dataframe
trials_df = optimizer.get_trials_dataframe()
if trials_df is not None:
    print("\nTop 5 trials:")
    display(trials_df.nlargest(5, 'value')[['number', 'value', 'params_rsi_overbought', 'params_rsi_oversold']])

In [ ]:
# Save best parameters
output_path = Path('../config/best_params.yaml')
optimizer.save_best_params(output_path)
print(f"Saved best parameters to {output_path}")